In [8]:
import matplotlib.pyplot as plt
from data_generation import MEANS, VARIANCES, KEY, generate_mv_data, NUM_SAMPLES
from plot_utils import plot_scatter, plot_circles, plot_metrics
from train_utils import SimpleDnn, Encoder, Linear_Encoder, create_loaders
import torch
from torch.utils.data import TensorDataset
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
import torch.nn.functional as F
import pandas as pd
from sklearn.datasets import make_circles
import matplotlib.image as image
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import plotly.express as px
from csv_utils import format_csv
from pathlib import Path

BATCH_SIZE = 32

# Linear Data Example

1. Generate linear data from three Gaussians

In [9]:
# generate data
data, labels = generate_mv_data(KEY, MEANS, VARIANCES, NUM_SAMPLES, 3)

# format labels and set up dataloaders
labels_one_hot = F.one_hot(torch.Tensor(np.hstack((np.array(labels)))).to(torch.int64), num_classes=3).float() 
data_linear = TensorDataset(
                torch.Tensor(np.vstack((np.array(data)))), labels_one_hot
            )
train_loader_linear, val_loader_linear = create_loaders(data = data_linear, ratio = 0.8, num_workers = 0, shuffle_train = True, shuffle_val = False, batch_size = 32)

2023-04-02 15:41:23.525 | INFO     | train_utils:create_loaders:43 - Creating dataloaders with 80/20train/test split 🔪
2023-04-02 15:41:23.526 | INFO     | train_utils:create_loaders:66 - Successfully created train and validation loader 🤗


### 1. Train 'unbroken' model (simple feed-forward network)

In [10]:
# define model and logger
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 3), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'linear')
# train model on linear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50, log_every_n_steps=20)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_linear, val_dataloaders = val_loader_linear)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory metrics_csv/linear/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 63    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
63        Trainable params
0         Non-trainable params
63        Total params
0.000     Total estimated model params size (MB)
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment log

Epoch 49: 100%|██████████| 38/38 [00:00<00:00, 177.47it/s, v_num=0]         

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 38/38 [00:00<00:00, 173.16it/s, v_num=0]


In [11]:
# format, since csv logger captures metrics on step basis
format_csv('metrics_csv/linear/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/linear_data.csv'))

### 2. Broken model: No activation functions

In [12]:
# DNN without ReLUs
simple_dnn_linear = SimpleDnn(Linear_Encoder(input_dim = 2, output_dim = 3), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'linear_no_relu')
# train model on linear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50, log_every_n_steps=20)
trainer.fit(model=simple_dnn_linear, train_dataloaders = train_loader_linear, val_dataloaders = val_loader_linear)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory metrics_csv/linear_no_relu/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type           | Params
-------------------------------------------
0 | encoder | Linear_Encoder | 63    
1 | f1      | BinaryF1Score  | 0     
-------------------------------------------
63        Trainable params
0         Non-trainable params
63        Total params
0.000     Total estimated model params size (MB)
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: E

/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 49: 100%|██████████| 38/38 [00:00<00:00, 152.62it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 38/38 [00:00<00:00, 149.26it/s, v_num=0]


In [13]:
# format, since csv logger captures metrics on step basis
format_csv('metrics_csv/linear_no_relu/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/linear_data_no_relu.csv'))

# Nonlinear Data Example

In [14]:
data_circles, label_circles = make_circles(n_samples=NUM_SAMPLES, factor=0.5, noise=0.05)
# concentric circles
data_tensor_circles = TensorDataset(
                torch.Tensor(data_circles), F.one_hot(torch.Tensor(label_circles).to(torch.int64), num_classes=2).float() 
            )
train_loader_circles, val_loader_circles = create_loaders(data = data_tensor_circles, ratio = 0.8,  batch_size = 32, num_workers = 0,
                                                        shuffle_train = True, shuffle_val = False)

2023-04-02 15:41:47.112 | INFO     | train_utils:create_loaders:43 - Creating dataloaders with 80/20train/test split 🔪
2023-04-02 15:41:47.114 | INFO     | train_utils:create_loaders:66 - Successfully created train and validation loader 🤗


In [15]:
### 1. Train 'unbroken' model (simple feed-forward network)

In [16]:
simple_dnn = SimpleDnn(Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles')
# train model on nonlinear data
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: metrics_csv/circles

  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 52    
1 | f1      | BinaryF1Score | 0     
------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model

Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 171.54it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 161.16it/s, v_num=0]


In [17]:
format_csv('metrics_csv/circles/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_data.csv'))

### 2. Broken model: No activation functions

In [18]:
# DNN without ReLUs
simple_dnn_linear = SimpleDnn(Linear_Encoder(input_dim = 2, output_dim = 2), task_type = 'classification')
# train model on nonlinear data
csv_logger = CSVLogger(save_dir='metrics_csv', name = 'circles_no_relu')
trainer = pl.Trainer(logger=csv_logger, max_epochs = 50)
trainer.fit(model=simple_dnn_linear, train_dataloaders = train_loader_circles, val_dataloaders = val_loader_circles)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: metrics_csv/circles_no_relu

  | Name    | Type           | Params
-------------------------------------------
0 | encoder | Linear_Encoder | 52    
1 | f1      | BinaryF1Score  | 0     
-------------------------------------------
52        Trainable params
0         Non-trainable params
52        Total params
0.000     Total estimated model params size (MB)


/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/clarahoffmann/Documents/code/pycon_2023_honey_i_broke_the_pytorch_model

Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 142.19it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:00<00:00, 132.34it/s, v_num=0]


In [20]:
format_csv('metrics_csv/circles_no_relu/version_0/metrics.csv', ['train_loss', 'val_loss'], Path('reformatted_metrics/circle_data_no_relu.csv'))